In [1]:
 current_dir = "gdrive/My Drive/Colab Notebooks/Thesis/gold_standard"


import os
import glob
from google.colab import drive
import pandas as pd
import re
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import re
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    RidgeCV,
    LassoCV,
    ElasticNetCV,
)


from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor


# Mount Google Drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [3]:
os.chdir(current_dir)

In [5]:
df = pd.read_csv("psychopathy_df.csv")

In [6]:
df

Unnamed: 0      name  score  \
0           0     wayne   40.0   
1           1      carl   36.0   
2           2     bundy   34.0   
3           3  bernardo   35.0   
4           4    dahmer   23.0   
5           5      gacy   27.0   
6           6    kemper   26.0   
7           7     marsh   34.5   
8           8  mitchell   34.0   
9           9  ridgeway   19.0   

                                                text  \
0  \nmy very first contemplation of criminal beha...   
1  I had a statutory rape on a minor fifteen uh t...   
2  \nyes that's true\n\ni grew up in a wonderful ...   
3  \npaul jason teal aka paul bernardo\n\n\nyeah ...   
4  \nhey you're lucky you came up on a\n\n\nday w...   
5  \n\nwell that's hogwash yeah that's strictly h...   
6  \nwell i'm not an expert i'm not an authority ...   
7  \nevery time i look at someone i see flashes o...   
8  that would be nice i'd like to use the restroo...   
9  \ni didn't really think i had a a urge to kill...   

                                          clean_text            binary  \
0   my very first contemplation of criminal behav...      psychopathic   
1  i had a statutory rape on a minor fifteen uh t...      psychopathic   
2   yes thats true i grew up in a wonderful home ...      psychopathic   
3   paul jason teal aka paul bernardo yeah im and...      psychopathic   
4   hey youre lucky you came up on a day when the...  not psychopathic   
5   well thats hogwash yeah thats strictly hogwas...  not psychopathic   
6   well im not an expert im not an authority im ...  not psychopathic   
7   every time i look at someone i see flashes of...      psychopathic   
8  that would be nice id like to use the restroom...      psychopathic   
9   i didnt really think i had a a urge to kill h...  not psychopathic   

  trinary     group                                                ADJ  \
0    high       doc  first criminal other little more dead opposite...   
1    high       doc  statutory minor sexual female old good female ...   
2    high  handbook  true wonderful dedicated young young local loc...   
3    high       cbc  peel regional public peel regional big crazy c...   
4  medium  handbook  lucky crazy slow steady ordinary last last int...   
5  medium  handbook  low last simple homosexual young correct sound...   
6  medium  handbook  more impossible nice ordinary most sick other ...   
7    high  articles  numerous numerous horrible terrible personal f...   
8    high  articles  nice only right almighty holy true holy other ...   
9     low  handbook  fourth fourth fourth fifth sixth angry alone a...   

                                                 ADP  ...  \
0  of of with on down up with of off on for besid...  ...   
1                       on for on on around of to at  ...   
2  up in with of as of outside in to in of out of...  ...   
3  about with of on for by around on to on with f...  ...   
4  up on like out of to about to of of off for of...  ...   
5  from in of against in on for to for for for of...  ...   
6  for than in up up in out of in as of on in out...  ...   
7  at of of in into with into into like at with o...  ...   
8  in of with with about through through of by of...  ...   
9  to in at for in of at for on during during out...  ...   

                                                 NUM  \
0     five twenty four four twelve forty five twenty   
1                                           four one   
2  two one five twelve thirteen one twenty thirty...   
3                          seventeen one one one two   
4  fourteen fifteen fifteen three eighteen five n...   
5  twelve nineteen seventy eight three million fo...   
6  twenty thirty five one one one seventy two one...   
7                                  two seventeen one   
8  ten eighteen fifty three eighteen eighteen eig...   
9  one one one one fourteen fifteen two two one o...   

                                                PART  \
0  to na to to nt to to to to to to 

In [ ]:
models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SVR(),
]
features = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB',]

In [ ]:
from itertools import combinations
feature_combos = [list(combo) for combo in combinations(features, 2)]

In [ ]:
columns_to_keep  = ['ADJ', 'ADP', 'ADV', 'AUX', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'SCONJ', 'VERB', 'group', 'name', "score"]

In [ ]:
data = df[columns_to_keep]

In [ ]:
data

ADJ  \
0  first criminal other little more dead opposite...   
1  statutory minor sexual female old good female ...   
2  true wonderful dedicated young young local loc...   
3  peel regional public peel regional big crazy c...   
4  lucky crazy slow steady ordinary last last int...   
5  low last simple homosexual young correct sound...   
6  more impossible nice ordinary most sick other ...   
7  numerous numerous horrible terrible personal f...   
8  nice only right almighty holy true holy other ...   
9  fourth fourth fourth fifth sixth angry alone a...   

                                                 ADP  \
0  of of with on down up with of off on for besid...   
1                       on for on on around of to at   
2  up in with of as of outside in to in of out of...   
3  about with of on for by around on to on with f...   
4  up on like out of to about to of of off for of...   
5  from in of against in on for to for for for of...   
6  for than in up up in out of in as of on in out...   
7  at of of in into with into into like at with o...   
8  in of with with about through through of by of...   
9  to in at for in of at for on during during out...   

                                                 ADV  \
0  very together away ever so just so right sure ...   
1                        then someday very at around   
2  certainly again more generally more more more ...   
3  aka forward maybe here just in back there here...   
4  billy up very just just very so thoroughly jus...   
5  strictly constantly constantly right always aw...   
6  almost far that even increasingly back further...   
7  just everywhere usually just there probably us...   
8  as absolutely so so now once again still very ...   
9  really just just probably probably too so just...   

                                                 AUX  \
0  was was s had would was do be is m are can be ...   
1                              was was be is is m be   
2  do would re might would is would is will is wa...   
3  did was did ve been was did did were can was a...   
4  was was was were were was do been been be is i...   
5  s m m ve were has been ve ve been ve had were ...   
6  has been would be is did did did m is was was ...   
7  ca do was should have been do is do do was was...   
8  would be may is can have am was did was ll was...   
9  did did was being was do was could was did mus...   

                                                 DET  \
0  a every the some that this a a a a the the the...   
1                               a a an a any a a all   
2  a a a a the the the the the an the the the a a...   
3  those the a the this that that that no the thi...   
4  a no all the the all all quite a some the some...   
5  a the this the no no the the the the a all tha...   
6  an an a a this the some the a an a an the the ...   
7  every the the the a the that an these the a th...   
8  the a a a a d the the the the the the the the ...   
9  a a the that no no that a the a a a a that no ...   

                                                INTJ  \
0  uh uh um uh uh uh okay uh well uh uh uh uh uh ...   
1                                              uh uh   
2  yes uh um um uh um uh uh uh uh um um uh uh uh ...   
3  yeah okay um uh uh well oh like well uh okay w...   
4  hey yeah uh uh yes yes uh uh uh uh uh uh uh uh...   
5  well yeah uh uh well uh uh like yeah well uh w...   
6  well uh uh yes uh uh no uh anyway uh oh my god...   
7  like uh uh like like yeah yeah yeah like well ...   
8  yes yes okay hello yes yeah yes okay okay well...   
9  uh um uh um um uh no um uh um um ah ah um uh u...   

                                                NOUN  \
0  contemplation behavior plan murder brother jum...   
1  rape fifteen assault child years child child t...   
2  home parents brothers sisters boy boy home gro...   
3  sense ones problem tv rolls announcement file ...   
4  day snow weekend book dad week night night sur...   
5  attorney

In [ ]:
X = data[features]
y = data["score"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

                                                 ADJ  \
0  first criminal other little more dead opposite...   
7  numerous numerous horrible terrible personal f...   
2  true wonderful dedicated young young local loc...   
9  fourth fourth fourth fifth sixth angry alone a...   
4  lucky crazy slow steady ordinary last last int...   
3  peel regional public peel regional big crazy c...   
6  more impossible nice ordinary most sick other ...   

                                                 ADP  \
0  of of with on down up with of off on for besid...   
7  at of of in into with into into like at with o...   
2  up in with of as of outside in to in of out of...   
9  to in at for in of at for on during during out...   
4  up on like out of to about to of of off for of...   
3  about with of on for by around on to on with f...   
6  for than in up up in out of in as of on in out...   

                                                 ADV  \
0  very together away ever so just so right su